In [11]:
import nltk
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
import pickle
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [12]:
import os
cwd = os.getcwd()
cwd

'/Users/apple/Documents/Kaggle/TextClassification'

In [13]:
data = pd.read_csv('/Users/apple/Documents/Kaggle/TextClassification/bbc-text.csv')
data.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2225 non-null   object
 1   text      2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


In [15]:
data["label"]=data["category"]
data["input"]= data["text"]

In [16]:
data.drop(["category","text"], axis =1,inplace = True)

In [17]:
data["label"].unique()

array(['tech', 'business', 'sport', 'entertainment', 'politics'],
      dtype=object)

In [18]:
data["label"].replace({"tech": 0, "business":1, "sport":2, "entertainment":3, "politics":4}, inplace=True)

In [19]:
# count vector for text
count_vect = CountVectorizer(analyzer = "word")
count_vec_X = count_vect.fit_transform(data["input"])

cvtrain_x,cvtest_x,cvtrain_y,cvtest_y = train_test_split(count_vec_X,data["label"],test_size = 0.2)

In [20]:
cvtrain_x

<1780x29421 sparse matrix of type '<class 'numpy.int64'>'
	with 358755 stored elements in Compressed Sparse Row format>

In [21]:
cvtrain_x.shape

(1780, 29421)

In [22]:
# tfidf vector for text on word level
tfidf_obj = TfidfVectorizer(analyzer = "word", max_features = 5000)
tfidf_vec_X = tfidf_obj.fit_transform(data["input"])
tfidf_train_x,tfidf_test_x,tfidf_train_y,tfidf_test_y = train_test_split(tfidf_vec_X,data["label"],test_size = 0.2)

In [23]:
# tfidf on ngram level
tfidf_ngram = TfidfVectorizer(analyzer = "word", ngram_range =(2,3),max_features = 5000)
tfidf_ngram_X = tfidf_ngram.fit_transform(data["input"])
ngram_train_x,ngram_test_x,ngram_train_y,ngram_test_y = train_test_split(tfidf_vec_X,data["label"],test_size = 0.2)

In [24]:
#utility function for model building
def train_model(model_classifier, train_x,test_x, train_y,test_y):
    model_classifier.fit(train_x,train_y)
    
    prediction = model_classifier.predict(test_x)
    
    print(prediction)
    
    return metrics.accuracy_score(prediction, test_y)
    

In [25]:
from sklearn import linear_model, naive_bayes, metrics, svm

# naivebayes model on count vector
accuracy_naiveBayes = train_model(naive_bayes.MultinomialNB(),cvtrain_x,cvtest_x,cvtrain_y,cvtest_y)
print("NB, Count Vectors: ", accuracy_naiveBayes)

# naivebayes model on tfidf vector
accuracy = train_model(naive_bayes.MultinomialNB(),tfidf_train_x,tfidf_test_x,tfidf_train_y,tfidf_test_y)
print("NB,Tfidf word level Vectors: ", accuracy)


# naivebayes model on tfidf ngram vector
accuracy = train_model(naive_bayes.MultinomialNB(),ngram_train_x,ngram_test_x,ngram_train_y,ngram_test_y)
print("NB,Tfidf ngram level Vectors: ", accuracy)

[2 1 2 2 1 4 4 3 1 4 3 3 3 0 2 2 2 3 2 1 2 1 0 0 0 2 2 2 3 4 1 0 4 1 4 2 2
 0 2 1 4 4 0 1 3 1 2 0 1 2 3 4 3 0 2 1 0 2 4 3 3 3 0 4 3 1 4 4 2 0 1 4 1 1
 0 2 1 0 1 0 4 2 2 1 0 1 3 2 4 3 2 0 2 1 0 1 0 4 1 4 1 3 1 4 1 0 4 0 1 3 2
 3 1 2 0 4 2 3 4 3 4 1 0 0 3 2 4 0 3 4 3 1 3 4 2 2 2 4 4 4 1 1 0 4 1 4 4 1
 1 3 1 2 0 4 1 2 3 1 1 4 2 0 2 4 3 2 1 0 0 1 3 3 2 2 3 4 2 3 3 1 1 2 3 4 0
 1 4 4 3 3 4 2 3 1 3 1 1 0 4 3 4 2 1 3 4 3 0 3 1 2 0 3 0 3 2 0 0 0 4 2 3 3
 3 2 2 1 2 3 2 4 2 3 1 2 0 1 0 2 2 2 4 4 2 2 2 3 4 1 2 4 1 1 0 2 3 0 1 3 1
 2 2 4 1 4 4 3 3 3 1 2 0 3 0 4 2 0 4 4 0 1 1 3 0 3 1 4 0 3 4 0 0 2 3 3 4 4
 3 4 2 1 3 2 2 0 0 0 1 0 1 4 2 2 4 4 1 0 2 2 2 4 1 4 0 0 4 2 0 0 2 2 4 1 1
 2 0 0 0 0 3 1 1 0 1 3 3 0 2 2 1 1 1 2 1 1 1 2 4 3 2 4 3 4 4 0 2 4 4 4 1 3
 1 4 3 3 4 2 3 1 1 0 2 1 0 2 1 2 2 2 4 3 3 3 1 1 1 2 1 0 0 3 4 1 1 2 2 4 0
 0 1 2 3 1 2 1 2 1 4 1 2 2 4 4 1 1 4 2 0 0 1 1 3 3 1 3 2 4 1 4 0 1 3 4 4 1
 4]
NB, Count Vectors:  0.9865168539325843
[2 1 1 3 4 1 2 0 1 3 3 2 1 3 3 0 3 2 2 1 4 4 1 3 2 1 1 4 

In [26]:
# Logistic Regression

accuracy = train_model(linear_model.LogisticRegression(),cvtrain_x,cvtest_x,cvtrain_y,cvtest_y)
print("Logistic regression,count Vectors: ", accuracy)

# Logistic Regression model on tfidf vector
accuracy = train_model(linear_model.LogisticRegression(),tfidf_train_x,tfidf_test_x,tfidf_train_y,tfidf_test_y)
print("Logistic Regression,Tfidf word level Vectors: ", accuracy)


# Logistic Regression model on tfidf ngram vector
accuracy = train_model(linear_model.LogisticRegression(),ngram_train_x,ngram_test_x,ngram_train_y,ngram_test_y)
print("Logistic Regression,Tfidf ngram Vectors: ", accuracy)

/usr/local/Cellar/jupyterlab/3.0.16/libexec/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[2 1 2 2 1 4 4 3 1 4 3 3 3 0 2 2 2 3 2 1 2 1 0 0 0 2 2 2 3 4 1 0 4 1 4 2 2
 0 2 1 4 4 0 1 3 1 2 1 1 2 3 4 3 0 2 1 0 2 4 3 3 3 0 4 3 1 4 4 2 0 1 4 1 3
 0 2 1 0 1 0 4 2 2 1 0 1 3 2 4 3 2 0 2 1 0 1 0 4 1 4 1 3 1 4 1 0 4 0 1 3 2
 3 1 2 0 4 2 3 4 3 4 1 0 0 3 2 4 0 3 4 3 1 3 4 2 2 2 4 4 4 1 1 0 4 1 4 4 1
 1 3 1 2 0 4 1 2 3 1 1 4 2 0 2 4 3 2 1 0 0 1 3 3 2 2 3 4 2 3 3 1 1 2 3 0 0
 1 4 4 3 3 4 2 3 1 3 1 1 0 4 3 4 3 1 3 4 3 0 3 1 2 0 3 0 3 2 0 0 0 4 2 3 3
 3 2 2 1 2 3 2 4 2 3 1 2 0 1 0 2 2 2 4 4 2 2 2 3 4 1 2 4 1 1 0 2 3 0 1 3 1
 2 2 4 1 4 4 3 3 3 1 2 0 3 0 4 2 0 4 4 0 1 1 3 0 3 1 4 0 3 4 0 0 2 3 3 4 4
 3 4 2 1 3 2 2 0 0 0 1 1 1 4 2 2 4 4 1 0 2 2 2 4 1 4 0 0 4 2 0 1 2 2 4 1 1
 2 0 0 0 0 3 1 1 0 1 3 3 0 2 2 1 1 1 2 1 1 1 2 4 3 2 4 3 4 4 0 2 4 4 4 1 3
 1 4 3 3 4 2 3 1 1 0 2 1 0 2 1 2 2 2 4 3 3 3 1 1 1 2 1 0 0 3 2 1 1 2 2 4 0
 0 1 2 3 1 2 1 2 1 4 1 2 2 0 4 1 1 4 2 0 0 1 1 3 3 1 4 2 4 1 4 0 1 3 4 4 1
 4]
Logistic regression,count Vectors:  0.9842696629213483
[2 1 1 3 4 1 2 0 1 3 3 2 1 3 3 0 3 2 2 1 

In [27]:
# support Vector machine on count vector
accuracy = train_model(svm.SVC(),cvtrain_x,cvtest_x,cvtrain_y,cvtest_y)
print("Support vector machine,count Vectors: ", accuracy)

# support Vector machine on tfidf vector
accuracy = train_model(svm.SVC(),tfidf_train_x,tfidf_test_x,tfidf_train_y,tfidf_test_y)
print("Support vector machine,Tfidf word level Vectors: ", accuracy)


# support Vector machine on tfidf ngram vector
accuracy = train_model(svm.SVC(),ngram_train_x,ngram_test_x,ngram_train_y,ngram_test_y)
print("Support vector machine,Tfidf ngram Vectors: ", accuracy)

[2 1 2 2 1 4 4 3 1 4 3 3 3 0 2 2 2 3 2 1 2 1 0 0 0 2 2 2 3 4 1 0 4 1 4 2 2
 0 2 1 2 4 0 1 3 1 2 1 1 2 3 4 3 0 2 1 0 2 4 3 3 3 0 4 3 1 4 4 2 0 1 4 1 3
 0 2 1 0 1 0 4 2 2 1 0 1 3 2 4 3 2 0 2 1 0 1 0 0 1 4 1 3 1 2 1 0 4 0 1 3 2
 3 1 2 0 4 2 3 4 3 4 1 0 0 3 2 4 0 3 4 3 1 3 4 2 2 2 4 4 4 1 1 0 4 1 4 2 1
 1 3 1 2 0 4 4 2 3 1 1 4 2 0 2 4 3 2 1 1 0 1 3 3 2 2 3 4 2 3 3 1 1 2 3 0 0
 1 4 4 3 3 4 2 3 1 3 1 1 0 4 3 4 3 1 3 4 2 0 3 1 2 0 3 0 3 2 0 0 0 2 2 3 3
 3 2 2 1 2 3 2 4 2 3 1 2 1 1 0 2 2 2 4 4 2 2 2 3 4 1 2 4 1 1 0 2 3 0 1 3 1
 2 2 4 1 4 4 3 3 3 1 2 0 3 0 4 2 0 4 4 0 1 1 3 0 3 1 4 0 3 4 0 0 2 3 3 4 4
 3 4 2 1 3 2 2 0 0 0 1 0 1 4 2 2 1 4 1 0 2 2 2 4 1 4 0 0 4 2 0 0 2 2 4 1 1
 2 0 0 0 0 3 1 1 0 3 3 3 0 2 2 1 1 1 2 1 1 1 2 4 3 2 4 3 4 4 0 2 4 4 4 1 3
 1 4 3 3 4 2 3 1 1 0 2 1 0 2 4 2 2 2 4 3 2 3 1 1 1 2 1 0 0 3 2 1 1 2 2 4 0
 0 1 2 3 1 2 1 2 1 4 1 2 2 4 4 1 1 4 2 1 0 1 1 3 1 1 0 2 4 1 4 0 1 3 4 4 1
 4]
Support vector machine,count Vectors:  0.952808988764045
[2 1 1 3 4 1 2 0 1 3 3 2 1 3 3 0 3 2 2 

In [28]:
with open ('input.txt', 'rt') as myfile:  # Open lorem.txt for reading text
    contents = myfile.read()              # Read the entire file to a string


contents

'Hutchins had been working as director of photography on the set of Rust.\n\nAmerican Cinematographer magazine had named her one of its rising stars in 2019, and she previously worked on 2020 independent superhero film Archenemy.\n\nArchenemy director Adam Egypt Mortimer told BBC News the fact she had died on a set was "really unbelievable".He said: "Halyna was an incredible artist who was just starting a career I think people were really starting to notice.\n\n"The fact that she would be killed on a set in an accident like this is unfathomable. It just seems inconceivable." Hutchins\' most recent post on Instagram, from Tuesday, showed her riding horses on set. On Twitter, Alec Baldwin said "there are no words to convey my shock and sadness regarding the tragic accident that took the life of Halyna Hutchins, a wife, mother and deeply admired colleague of ours."\n\n"My heart is broken for her husband, their son, and all who knew and loved Halyna," he added.Fellow cinematographer Cather

In [29]:
with open ('input.txt', 'rt') as myfile:  # Open lorem.txt for reading text
    contents = myfile.read()              # Read the entire file to a string
   


text = [contents]
data = {'input':text}

dframe = pd.DataFrame(data)
dframe


#utility function for prediction



model = naive_bayes.MultinomialNB()
model.fit(cvtrain_x, cvtrain_y)

vec2 = count_vect.transform(dframe["input"])

y_prediction = model.predict(vec2)
y_prediction

arr = y_prediction

li = [arr]
pd.DataFrame(data = li, columns= ["output"])

,output
0,3


In [30]:
vec2

<1x29421 sparse matrix of type '<class 'numpy.int64'>'
	with 126 stored elements in Compressed Sparse Row format>

In [24]:
filename = 'docClassficationModel.pkl'
pickle.dump(model, open(filename, 'wb'))

In [61]:
# Save the vectorizer
vec_file = 'vectorizer.pickle'
pickle.dump(count_vect, open(vec_file, 'wb'))

In [23]:
Pred

0.9842696629213483